# Local Variational Methods

An alternative 'local' approach to a variational approximation which involves finding bounds on functions over individual variables or groups of variables within a model

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
%config InlineBackend.figure_format = "retina"
plt.rcParams["figure.figsize"] = (8, 4)

## A first example $f(x) = \exp(-x)$

$$
    f(x) = \max_{\eta}\left\{\eta x - \eta + \eta\log(-\eta)\right\}
$$

In [3]:
def f(x):
    return np.exp(-x)


def f_linear(x, η):
    """
    First order linear approximation to exp(-x)
    
    Parameters
    ----------
    x: float
        Evaluation point
    η: float
        η := -exp(-x)
    """
    return η * x - η + η * np.log(-η)


def f_prime(f, x, eps=1e-8):
    return (f(x + eps) - f(x)) / eps


def gradient_descent(f, xinit, eps=1e-5, alpha=0.01):
    delta = np.inf
    x_star = xinit
    while delta > eps:
        grad_eta = f_prime(f, x_star)
        x_star_new = x_star - alpha * grad_eta
        delta = abs(x_star_new / x_star - 1)
        x_star = x_star_new
    
    return x_star

In [4]:
xlower = np.linspace(-10, 10)
etas = np.arange(-0.1, -10, -0.2)
xrange = -4 + 10 ** np.linspace(0, 1)
    
eta_init, alpha = -0.1, 1e-3
eta_collection = []
for x in xrange[::2]:
    eta_hat, _ = gradient_descent(lambda eta: -f_linear(x, eta), eta_init, alpha=alpha)
    plt.scatter(x, f_linear(x, eta_hat), c="tab:red")
    eta_collection.append(eta_hat)
    
for eta in eta_collection:
    plt.plot(xlower, f_linear(xlower, eta), alpha=0.5, c="tab:gray")

    
plt.plot(xrange, f(xrange), linewidth=2)

plt.ylim(-10, 20)
eq = r"$\exp(-x) = \max_{\eta}\left\{\eta x - \eta + \eta\log(-\eta)\right\}$"
plt.title(eq + "\nTangent Approximation", fontsize=15)
plt.grid()

TypeError: cannot unpack non-iterable numpy.float64 object